In [1]:
%%capture
! pip install datasets fast-fit transformers accelerate
! pip install evaluate
! pip install langdetect

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
USE_COLAB = True

In [5]:
import os
import evaluate
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline,
)
from datasets import Dataset

if USE_COLAB:
    from drive.MyDrive.Github.NLPSharedTask.essentials.config import ABSTRACTS
    from drive.MyDrive.Github.NLPSharedTask.essentials.data_functions import read_data
else:
    from essentials.config import ABSTRACTS
    from essentials.data_functions import read_data

In [7]:
# Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

label2id={n: n+1 for n in range(0,17)}
id2label={n: n-1 for n in range(1,18)}

# Define model
model = AutoModelForSequenceClassification.from_pretrained(
    'allenai/scibert_scivocab_uncased',
    num_labels=18,
    return_dict=True)

# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
if USE_COLAB:
    base_dir = 'drive/MyDrive/Github/NLPSharedTask'
else:
    base_dir = ''

# LOAD DATA

In [10]:
df = pd.read_csv(os.path.join(base_dir, 'cleaned_data_with_null_with_weakly_labeled_with_synth.csv'))

In [ ]:
def balance_classes(df, target_col, balance_level=0.5):
    """
    Balance the class distribution in a DataFrame to reduce class imbalances.

    Args:
    df (pd.DataFrame): The DataFrame containing the data.
    target_col (str): The name of the column that contains the target classes.
    balance_level (float, optional): A float between 0 and 1 where 0 means no balancing and 1 means full balancing. Default is 0.5.

    Returns:
    pd.DataFrame: A DataFrame with adjusted class distribution.
    """
    if not (0 <= balance_level <= 1):
        raise ValueError("Balance level must be between 0 and 1.")

    # Calculate class counts and determine the maximum number we want for each class
    class_counts = df[target_col].value_counts()
    max_size = class_counts.min() + (class_counts.max() - class_counts.min()) * balance_level

    # Sample from each class
    def resample_class(group):
        n = min(len(group), int(max_size))
        return group.sample(n, replace=False)

    # Group by the target column and apply sampling
    balanced_df = df.groupby(target_col).apply(resample_class).reset_index(drop=True)

    return balanced_df

# CREATE TRAIN/TEST SPLIT

In [17]:
def tokenize_text(texts):
    return tokenizer(texts, truncation=True, max_length=256, return_tensors=None)

In [18]:
def rule_based_train_test_split(
    data: pd.DataFrame,
    label_col: str = 'label',
    test_size: float = 0.3,
    random_state: int | None = None
) -> dict:
    """Creates train-test split that makes sure that at least two abstracts for each id are in the test set."""

    abstract_data = data[data.is_abstract == 1]

    # Randomly sample 2 abstracts per sdg group
    test_a = abstract_data.groupby(label_col).sample(n=1, random_state=random_state)

    # Remove the entries already in the test set from the rest of the data
    data = data[~data.index.isin(test_a.index)].copy()

    # Split the remaining data into train and test
    train, test_b = train_test_split(data, test_size=test_size, random_state=random_state, stratify=data[label_col])

    # Concatenate both test sets and shuffle them again
    test = pd.concat([test_a, test_b]).sample(frac=1).reset_index(drop=True)

    return train, test

In [19]:
# Apply huggingface tokenizer
tokenized_output = tokenize_text(df['text_clean'].to_list())

In [21]:
df_tokenized = pd.DataFrame({
    'raw_text': df['text_clean'].tolist(),
    'input_ids': list(tokenized_output['input_ids']),
    'attention_mask': list(tokenized_output['attention_mask']),
    'token_type_ids': list(tokenized_output.get('token_type_ids', [[]]*len(df))),
    'label': df['label'].tolist(),
    'is_abstract': df['is_abstract'].to_list()
})

train_df, test_df = rule_based_train_test_split(df_tokenized, random_state=42)

train_df.to_csv(os.path.join(base_dir, 'train_df.csv'))
test_df.to_csv(os.path.join(base_dir, 'test_df.csv'))

train_dataset = Dataset.from_pandas(train_df[['input_ids', 'attention_mask', 'token_type_ids', 'label']])
test_dataset = Dataset.from_pandas(test_df[['input_ids', 'attention_mask', 'token_type_ids', 'label']])

# FINE-TUNING

For training, use the suggested values from the paper:

In all settings, we apply a dropout of 0.1 and optimize cross entropy loss using Adam (Kingma and Ba, 2015). We finetune for 2 to 5 epochs using a batch size of 32 and a learning rate of 5e-6, 1e-5, 2e-5, or 5e-5 with a slanted triangular schedule (Howard and Ruder, 2018) which is equivalent to the linear warmup followed by linear decay (Devlin et al., 2019). For each dataset and BERT variant, we pick the best learning rate and number of epochs on the development set and report the corresponding test results. We found the setting that works best across most datasets and models is 2 or 4 epochs and a learning rate of 2e-5. While task-dependent, optimal hyperparameters for each task are often the same across BERT variants.

In [22]:
# Multiple class prediction (one prediction)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    }

In [23]:
# Training arguments
training_args = TrainingArguments(
    output_dir=os.path.join(base_dir, 'models/results'),
    num_train_epochs=2,  # As best setting suggested 2 or 4
    warmup_steps=500,  # Slanted triangular schedule start
    learning_rate=2e-5,  # Best learning rate as suggested in the paper
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    lr_scheduler_type='linear',  # Corresponds to linear warmup followed by linear decay
)

In [24]:
# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Adam Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Multiple class Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

In [25]:
# Start training
torch.cuda.empty_cache()
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.780600,0.755494,0.791749,0.792058
2,0.561100,0.676200,0.815167,0.814943


TrainOutput(global_step=10652, training_loss=0.8309846721195073, metrics={'train_runtime': 2379.6651, 'train_samples_per_second': 35.805, 'train_steps_per_second': 4.476, 'total_flos': 5129265581828568.0, 'train_loss': 0.8309846721195073, 'epoch': 2.0})

In [26]:
# Saving the model
model_path = os.path.join(base_dir, 'models/scibert_model_with_null_with_synth_with_weakly_labeled')
trainer.save_model(model_path)

# Saving the tokenizer associated with the model
tokenizer.save_pretrained(model_path)

('drive/MyDrive/Github/NLPSharedTask/models/scibert_model_with_null_with_synth_with_weakly_labeled/tokenizer_config.json',
 'drive/MyDrive/Github/NLPSharedTask/models/scibert_model_with_null_with_synth_with_weakly_labeled/special_tokens_map.json',
 'drive/MyDrive/Github/NLPSharedTask/models/scibert_model_with_null_with_synth_with_weakly_labeled/vocab.txt',
 'drive/MyDrive/Github/NLPSharedTask/models/scibert_model_with_null_with_synth_with_weakly_labeled/added_tokens.json',
 'drive/MyDrive/Github/NLPSharedTask/models/scibert_model_with_null_with_synth_with_weakly_labeled/tokenizer.json')

In [ ]:
# Load the trained model
model = AutoModelForSequenceClassification.from_pretrained(os.path.join(base_dir, 'models/scibert_model_base'))

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(os.path.join(base_dir, 'models/scibert_model_base'))

# Create a prediction pipeline
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
test_df = pd.read_csv(os.path.join(base_dir, 'test_df.csv'))

In [ ]:
test_df['prediction'] = test_df.raw_text.apply(nlp)

In [ ]:
test_df.to_csv(os.path.join(base_dir, 'test_df_with_weakly_labeled_data.csv'))